In [ ]:
import pandas as pd

tatortreviews = pd.read_csv('data/tatort_reviews_labeled.csv')
print("Number of reviews: ", len(tatortreviews))

tatortreviews.sort_values(by='Review')

630


,Review,Evaluation
116,- 6. wirklich übel 🤢,0
84,... von der nicht so wahnsinnig glücklichen Th...,0
98,.....klischeehaft wurde nahezu alle Aspekte de...,0
281,...endlich wieder mal ein Tatort der richtig K...,1
559,...geschaltet!,0
531,...sau blöder tatort......,0
613,1\nDas mit 1-6 müssen hier viele Pisas noch ma...,1
422,"1\nEin sehr komplexer, tragischer Fall 👍",1
181,1 Plus mit Sternchen,1
334,1 ! . ....endlich mal wieder 👍,1
